In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os


zip_path = "/content/drive/MyDrive/GIGGLE/dataset.zip"
extract_dir = "/content/drive/MyDrive/GIGGLE/behavior dataset"

# Unzipping the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Dataset unzipped successfully!")


Dataset unzipped successfully!


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameters
img_size = 128
num_classes = 2
batch_size = 32
epochs = 10

# Load dataset
def load_dataset(dataset_path="/content/drive/MyDrive/GIGGLE/behavior dataset"):
    images = []
    labels = []

    for label, folder in enumerate(["not_crossing", "crossing"]):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (img_size, img_size))
            images.append(img)
            labels.append(label)

    images = np.array(images, dtype=np.float32) / 255.0  # Normalize images
    labels = np.array(labels)

    return images, labels

# Load data
images, labels = load_dataset()

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# One-hot encode labels
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(X_train)

# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_val, y_val),
    epochs=epochs,
    steps_per_epoch=len(X_train) // batch_size,
    verbose=1
)

# Evaluate model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation accuracy: {val_accuracy:.2f}")


In [ ]:
# Save the model
model.save("/content/drive/MyDrive/GIGGLE/behavior dataset/crossing_classifier.h5")
print("Model saved successfully!")


Model saved successfully!


In [ ]:
from tensorflow.keras.models import load_model


loaded_model = load_model("/content/drive/MyDrive/GIGGLE/behavior dataset/crossing_classifier.h5")
print("Model loaded successfully!")


Model loaded successfully!


In [ ]:
def predict_image(image_path, model, img_size=128):
    # Load and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_size, img_size))
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize pixel values

    # Predict using the model
    prediction = model.predict(img)
    predicted_label = np.argmax(prediction)

    # Return the result
    return "Crossing" if predicted_label == 1 else "Not Crossing"

# Test prediction on a new image
image_path = "/content/drive/MyDrive/GIGGLE/Test/test1.jpg"
result = predict_image(image_path, loaded_model)
print(f"Prediction: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step
Prediction: Not Crossing


In [ ]:
image_path = "/content/drive/MyDrive/GIGGLE/Test/test 2.jpg"
result = predict_image(image_path, loaded_model)
print(f"Prediction: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prediction: Crossing
